In [1]:
import pandas as pd

In [2]:
data_old = pd.read_excel('Data\data_after_5_years.xlsx', sheet_name=0, header=1, index_col=1)
data_new = pd.read_excel('Data\data_after_5_years.xlsx', sheet_name=1, header=1, index_col=0)

In [3]:
data_old.rename_axis('index', inplace=True)
data_new.rename_axis('index', inplace=True)

In [4]:
def decode(data, new=False):
    diff_col = ['Наслед. по АГ', 'АГч/з 5лет'] if new else ['Наслед. по АГ(мать-1;отец-2;оба-3; нет-0)', 'АГ ст.']
    data_decode = data[['пол', 'возр', 'ИМТ(до 30,5-0;более 30,6-1)',
                        'ОТ.1', 'Физ акт(1,2-0);(3,4-1)', 'кур-ие(1,2-1);(3-0)',
                        'Уп. Алк(1,2-0);(3,4-1)'] + diff_col].copy()
    rename_dict = {'пол': 'gender',
                   'возр': 'age',
                   'АГ ст.': 'stage',
                   'АГч/з 5лет': 'stage',
                   'ИМТ(до 30,5-0;более 30,6-1)': 'bmi',
                   'ОТ.1': 'ao', 
                   'Физ акт(1,2-0);(3,4-1)': 'activity',
                   'кур-ие(1,2-1);(3-0)': 'smoking',
                   'Уп. Алк(1,2-0);(3,4-1)': 'alcohol',
                   'Наслед. по АГ(мать-1;отец-2;оба-3; нет-0)': 'male_heredity',
                   'Наслед. по АГ': 'male_heredity'}
    data_decode.rename(columns=rename_dict, inplace=True)
    data_decode.gender = data_decode.gender.map(lambda x: 1 if x=='м' else 0)
    data_decode.age = data_decode.age.map(lambda x: 1 if x>=45 else 0)
    data_decode.stage = data_decode.stage.map(lambda x: 0 if x=='зд' else 1)
    data_decode.male_heredity = data_decode.male_heredity.map(lambda x: 1 if x in [2, 3] else 0)
    data_decode.fillna(0, inplace=True)
    data_decode = data_decode.apply(lambda x: pd.to_numeric(x, downcast='integer'), axis=1)
    #data_decode.smoking = pd.to_numeric(data_decode.smoking.fillna(0), downcast='integer')
    #data_decode.activity = pd.to_numeric(data_decode.activity.fillna(0), downcast='integer')
    #data_decode.alcohol = pd.to_numeric(data_decode.alcohol.fillna(0), downcast='integer')
    data_decode.activity = data_decode.activity.map(lambda x: 0 if x==1 else 1)
    
    # data_decode['AGT_AGTR2'] = list(map(lambda x, y: 1 if x == 2 or 
    #                                    x == 3 and y == 3 else 0,
    #                                    data_decode.AGT, data_decode.AGTR2))
    # data_decode.drop(['AGT', 'AGTR2'], axis=1, inplace=True)
    return data_decode

In [5]:
decode(data_old).to_csv('Data\data_decode_old.csv')

In [6]:
decode(data_new, new=True).to_csv('Data\data_decode_new.csv')